# NiN
NIN结构的mlp conv结构是将多层感知机（MLP）和卷积(CONV)结合在一起，即在传统的卷积层之间加上了MLP层，一个卷积核对原图的同一片相同大小区域卷积得到一个数，96个卷积核同样的操作得到96个数，即96个通道上同一个位置的一个元素，然后将这96个元素做一下全连接，又得到96个元素。相当于在对每一个patch的不同卷积核结果做了一个MLP。

## 导入库

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

## 参数

In [2]:
BATCH_SIZE = 10000
EPOCH = 5

## 加载数据并调整大小

In [3]:
class DataLoader():
    def __init__(self):
        (self.train_images, self.train_labels), (self.test_images, self.test_labels) = keras.datasets.fashion_mnist.load_data()
        self.train_images = np.expand_dims(self.train_images.astype(np.float32)/255.0, axis=-1)
        self.test_images = np.expand_dims(self.test_images.astype(np.float32)/255.0, axis=-1)
        self.train_labels = self.train_labels.astype(np.int32)
        self.test_labels = self.test_labels.astype(np.int32)
        self.num_train, self.num_test = self.train_images.shape[0], self.test_images.shape[0]
        
    def get_batch_train(self, batch_size):
        index = np.random.randint(0, np.shape(self.train_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.train_images[index], 224, 224,)
        return resized_images.numpy(), self.train_labels[index]
    
    def get_batch_test(self, batch_size):
        index = np.random.randint(0, np.shape(self.test_images)[0], batch_size)
        resized_images = tf.image.resize_with_pad(self.test_images[index], 224, 224,)
        return resized_images.numpy(), self.test_labels[index]

In [4]:
dataLoader = DataLoader()
x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)

## 网络中的网络

In [5]:
def nin_block(num_channels, kernel_size, strides, padding):
    blk = keras.models.Sequential()
    blk.add(keras.layers.Conv2D(num_channels, kernel_size, strides=strides, padding=padding, activation='relu')) 
    blk.add(keras.layers.Conv2D(num_channels, kernel_size=1, activation='relu')) 
    blk.add(keras.layers.Conv2D(num_channels, kernel_size=1, activation='relu'))    
    return blk

## 定义总模型

In [6]:
model = keras.models.Sequential()

model.add(nin_block(96, kernel_size=11, strides=4, padding='valid'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2))

model.add(nin_block(256, kernel_size=5, strides=1, padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2))

model.add(nin_block(384, kernel_size=3, strides=1, padding='same'))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2))
model.add(keras.layers.Dropout(0.5))

model.add(nin_block(10, kernel_size=3, strides=1, padding='same'))

model.add(keras.layers.GlobalAveragePooling2D())
model.add(keras.layers.Flatten())

## 优化器和损失函数 

In [7]:
optimizer = keras.optimizers.Adam(lr=1e-7)
model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## 训练

In [8]:
num_iter = dataLoader.num_train//BATCH_SIZE
for e in range(EPOCH):
    for n in range(num_iter):
        x_batch, y_batch = dataLoader.get_batch_train(BATCH_SIZE)
        model.fit(x_batch, y_batch)

Train on 10000 samples
10000/10000 [==============================] - 18s 2ms/sample - loss: 2.6610 - accuracy: 0.1005s - loss: 2.6669 
Train on 10000 samples
10000/10000 [==============================] - 15s 2ms/sample - loss: 2.4812 - accuracy: 0.0942
Train on 10000 samples
10000/10000 [==============================] - 15s 2ms/sample - loss: 2.4328 - accuracy: 0.0883s - loss: 2.4397 - accura
Train on 10000 samples
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.3753 - accuracy: 0.1095s - loss: 2
Train on 10000 samples
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.3296 - accuracy: 0.1236s - loss: 2.3304 - accu
Train on 10000 samples
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.2892 - accuracy: 0.1415s -
Train on 10000 samples
10000/10000 [==============================] - 16s 2ms/sample - loss: 2.2434 - accuracy: 0.1593
Train on 10000 samples
10000/10000 [==============================] - 16s 2ms/sample - l

## 测试

In [9]:
x_test, y_test = dataLoader.get_batch_test(2000)
model.evaluate(x_test, y_test, verbose=2)

2000/2000 - 2s - loss: 1.6568 - accuracy: 0.3390


[1.6567667217254638, 0.33900002]